In [1]:
%load_ext sql

import pandas as pd
import numpy as np
import pybaseball
from pybaseball import statcast
import sqlite3

import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
#example usage
%sql sqlite:///../data/databases/mlb.db

#result = %sql SELECT * FROM employees WHERE salary > 50000;
#dataframe = result.DataFrame()

%sql select * from Statcast limit 10;
%sql select sql from sqlite_schema where name='Statcast';

 * sqlite:///../data/databases/mlb.db
Done.
 * sqlite:///../data/databases/mlb.db
Done.


sql
"CREATE TABLE Statcast (pitch_type TEXT, game_date TEXT, release_speed REAL, release_pos_x REAL, release_pos_z REAL, player_name TEXT, batter INTEGER, pitcher INTEGER, events TEXT, description TEXT, spin_dir INTEGER, spin_rate_deprecated INTEGER, break_angle_deprecated INTEGER, break_length_deprecated INTEGER, zone INTEGER, des TEXT, game_type TEXT, stand TEXT, p_throws TEXT, home_team TEXT, away_team TEXT, type TEXT, hit_location INTEGER, bb_type TEXT, balls INTEGER, strikes INTEGER, game_year INTEGER, pfx_x REAL, pfx_z REAL, plate_x REAL, plate_z REAL, on_3b INTEGER, on_2b INTEGER, on_1b INTEGER, outs_when_up INTEGER, inning INTEGER, inning_topbot TEXT, hc_x REAL, hc_y REAL, tfs_deprecated INTEGER, tfs_zulu_deprecated INTEGER, fielder_2 INTEGER, umpire INTEGER, sv_id INTEGER, vx0 REAL, vy0 REAL, vz0 REAL, ax REAL, ay REAL, az REAL, sz_top REAL, sz_bot REAL, hit_distance_sc INTEGER, launch_speed REAL, launch_angle INTEGER, effective_speed REAL, release_spin_rate INTEGER, release_extension REAL, game_pk INTEGER, fielder_3 INTEGER, fielder_4 INTEGER, fielder_5 INTEGER, fielder_6 INTEGER, fielder_7 INTEGER, fielder_8 INTEGER, fielder_9 INTEGER, release_pos_y REAL, estimated_ba_using_speedangle REAL, estimated_woba_using_speedangle REAL, woba_value REAL, woba_denom INTEGER, babip_value INTEGER, iso_value INTEGER, launch_speed_angle INTEGER, at_bat_number INTEGER, pitch_number INTEGER, pitch_name TEXT, home_score INTEGER, away_score INTEGER, bat_score INTEGER, fld_score INTEGER, post_away_score INTEGER, post_home_score INTEGER, post_bat_score INTEGER, post_fld_score INTEGER, if_fielding_alignment TEXT, of_fielding_alignment TEXT, spin_axis INTEGER, delta_home_win_exp REAL, delta_run_exp REAL, PRIMARY KEY (game_date, game_pk, at_bat_number, pitch_number))"


In [5]:
torch.cuda.is_available()

True

In [7]:
test_statcast = statcast(start_dt="2019-06-24", end_dt="2019-06-25")

This is a large query, it may take a moment to complete


  0%|                                                     | 0/2 [00:00<?, ?it/s]/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 50%|██████████████████████▌                      | 1/2 [00:07<00:07,  7.75s/it]/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|█████████████████████████████████████████████| 2/2 [00:15<00:00,  7.82s/it]


In [24]:
df = test_statcast.iloc[0:10]
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name',
       'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
       'post_home_score', 'post_bat_score', 'post_fld_score',
       'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis',
       'delta_home_win_exp', 'delta_run_exp'],
      dtype='object')

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [28]:
(df['pitcher.1'] == df['pitcher']).unique()
(df['fielder_2.1'] == df['fielder_2']).unique()

<BooleanArray>
[True]
Length: 1, dtype: boolean

In [21]:
df.apply(lambda row: print(row), axis=1)

pitch_type                                FF
game_date                2019-06-25 00:00:00
release_speed                           93.1
release_pos_x                          -2.33
release_pos_z                           5.21
                                ...         
if_fielding_alignment               Standard
of_fielding_alignment               Standard
spin_axis                                210
delta_home_win_exp                       0.0
delta_run_exp                         -0.115
Name: 2939, Length: 92, dtype: object
pitch_type                                SL
game_date                2019-06-25 00:00:00
release_speed                           87.0
release_pos_x                          -2.27
release_pos_z                           5.16
                                ...         
if_fielding_alignment               Standard
of_fielding_alignment               Standard
spin_axis                                151
delta_home_win_exp                    -0.002
delta_run_exp    

2939    None
3056    None
3222    None
3279    None
3418    None
3569    None
3632    None
3806    None
3945    None
4061    None
dtype: object

In [9]:
list(test_statcast.columns)

['pitch_type',
 'game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'player_name',
 'batter',
 'pitcher',
 'events',
 'description',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
 'game_type',
 'stand',
 'p_throws',
 'home_team',
 'away_team',
 'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'inning_topbot',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'fielder_2',
 'umpire',
 'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimat

In [6]:
test_statcast[['game_date', 'game_pk', 'at_bat_number', 'pitch_number']]

,game_date,game_pk,at_bat_number,pitch_number
2970,2019-06-25,567591,85,1
3040,2019-06-25,567591,84,6
3184,2019-06-25,567591,84,5
3351,2019-06-25,567591,84,4
3388,2019-06-25,567591,84,3
...,...,...,...,...
1861,2019-06-24,564869,2,1
1910,2019-06-24,564869,1,4
1928,2019-06-24,564869,1,3
2035,2019-06-24,564869,1,2


In [7]:
test_statcast.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6439 entries, 2970 to 2064
Data columns (total 92 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   pitch_type                       6436 non-null   object        
 1   game_date                        6439 non-null   datetime64[ns]
 2   release_speed                    6436 non-null   Float64       
 3   release_pos_x                    6436 non-null   Float64       
 4   release_pos_z                    6436 non-null   Float64       
 5   player_name                      6439 non-null   object        
 6   batter                           6439 non-null   Int64         
 7   pitcher                          6439 non-null   Int64         
 8   events                           1662 non-null   object        
 9   description                      6439 non-null   object        
 10  spin_dir                         0 non-null      Int64        